In [1]:
print('Importing Librairies...')
import pandas as pd
import time
from colorama import Fore, Back, Style
# XGBoost on Stock Price dataset, Tune n_estimators and max_depth
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,roc_curve,confusion_matrix,classification_report
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import warnings
import pickle
warnings.filterwarnings("ignore")
#import tensorflow as tf
#tf.logging.set_verbosity(tf.logging.ERROR)
print('Librairies imported.')
print('')

Importing Librairies...
Librairies imported.



In [2]:
LaDate = '2020-01-15'
ticker = 'USDJPY'
indice = 0.5
try:
    os.mkdir('DL_'+ticker+'-'+LaDate)
except:
    pass
try:
    os.mkdir('Boost_'+ticker+'-'+LaDate)
except:
    pass

In [3]:
# PARAMETRES TEMPORELS INITIAUX
start = '2003-05-04'
end = LaDate
print('DATE ORIGINELLE DU DEBUT DE TOUT... :',start)
error = []

DATE ORIGINELLE DU DEBUT DE TOUT... : 2003-05-04


In [4]:
print(' - Date de début : ',start,' - Date de fin : ',end)
print(Fore.YELLOW,'GENERATION DE LaBase :',Fore.MAGENTA,time.strftime("%H:%M:%S", time.localtime()))
tmps55=time.time()
LaBase = pd.read_csv('USDJPY.csv')
df = LaBase

INDEX = []
for i in range(0,len(df)):
    INDEX.append((str(df.iloc[i,0])[:2]+'-'+str(df.iloc[i,0])[3:5]+'-'+str(df.iloc[i,0])[6:10]))
df.index = INDEX
df = df.drop(['Local time'],axis=1)

delta = round(df.shape[0])
bloc1 = round(delta*0.75)
bloc2 = delta - bloc1

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for complete Base Preparation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
#LaBasePrep.to_csv('LaBasePrep.csv')
df.head()

 - Date de début :  2003-05-04  - Date de fin :  2020-01-15
 GENERATION DE LaBase :  20:11:57
 Time for complete Base Preparation = 0.140000 seconds 
 


,Open,High,Low,Close,Volume
04-05-2003,118.940,119.046,118.461,118.603,592866.9024
05-05-2003,118.591,118.751,117.290,117.500,581706.9990
06-05-2003,117.456,117.830,116.052,116.303,584496.2120
07-05-2003,116.311,116.969,115.940,116.823,588236.7032
08-05-2003,116.835,117.612,116.794,117.151,583132.8985


In [ ]:
print(Fore.YELLOW,'GENERATION DU BOOST :',Fore.MAGENTA,time.strftime("%H:%M:%S", time.localtime()))
tmps55=time.time()
X = df.copy()
X = X.drop(['Close'],axis=1)
X['Close'] = df['Close']
y = X.iloc[:,-1]
Xtrain = X.iloc[:-2,:-1]
Xtest = X.iloc[-2:-1,:-1]
yshift = y.shift(-1)
ytrain = yshift.iloc[:-2]
ytest = yshift.iloc[-2:-1]

#model = xgb.XGBRegressor(n_estimators=20000, learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100,objective='reg:squarederror')

#model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

#ytrain_pred = model.predict(Xtrain)

#y_pred = model.predict(Xtest)

#pred = model.predict(X.iloc[:,:-1])
#pickle.dump(model, open('Boost_'+ticker+'-'+LaDate+'/'+ticker+'_boost.dat', "wb"))
#df['Close.S'] = pred
#tmps22=round(time.time()-tmps55,2)
#print(Fore.BLUE,"Time for complete BOOST GENERATION = %f" %tmps22,'seconds \n',Style.RESET_ALL)




params = {'n_estimators':[100,100000],'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(1,6)],  'subsample':[i/10.0 for i in range(1,11)],
'colsample_bytree':[i/10.0 for i in range(1,11)], 'max_depth': [1,10,50,100,500]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1,objective='reg:squarederror') 

grid = GridSearchCV(xgb, params, verbose=1)
grid.fit(Xtrain, ytrain)
print('Done.')

 GENERATION DU BOOST :  20:11:59
Fitting 3 folds for each of 10000 candidates, totalling 30000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
from sklearn.metrics import mean_squared_error,r2_score
mse = mean_squared_error(ytest, grid.predict(Xtest))
print("MSE: %.4f" % mse)

# Print the r2 score
print(r2_score(ytest, grid.best_estimator_.predict(Xtest))) 